### Q1. Answer: 19

In [3]:
import pandas as pd


In [6]:
!pip install pyarrow


     |████████████████████████████████| 42.1 MB 22.4 MB/s eta 0:00:01


In [15]:
df = pd.read_parquet('./yellow_tripdata_2023-01.parquet')

In [8]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [9]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [13]:
len(df.columns)


19

### Q2. Answer: 42.59


In [20]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [25]:
# Duration column in seconds
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [30]:
# Deviation
std_duration = df['duration'].std()
print("Standard deviation of duration in January:", std_duration)

Standard deviation of duration in January: 42.594351241920904


### Q3. Answer: 98%

In [36]:
# Filter out outliers
mask = (df['duration'] >= 1) & (df['duration'] <= 60)
filtered_data = df[mask]

# Compute the fraction of remaining records
fraction_left = len(filtered_data) / len(df)
print("Fraction of remaining records:", fraction_left * 100)

Fraction of remaining records: 98.1220282212598


### Q4. Answer: 518

In [39]:
from sklearn.feature_extraction import DictVectorizer

In [40]:
# Preprocessing Pickup and Dropoff IDs
df['PULocationID'] = df['PULocationID'].astype(str)
df['DOLocationID'] = df['DOLocationID'].astype(str)

In [41]:
# Convert dataframe to list of dictionaries
features = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')


In [43]:
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(features)

In [44]:
# Output the number of columns in the feature matrix
print("Dimensionality of feature matrix:", X_train.shape[1])

Dimensionality of feature matrix: 518


In [45]:
from sklearn.feature_extraction import DictVectorizer

# Combine PULocationID and DOLocationID (optional step)
df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

# Ensure categorical columns are strings
df['PULocationID'] = df['PULocationID'].astype(str)
df['DOLocationID'] = df['DOLocationID'].astype(str)

# Create feature dictionaries
features = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Use DictVectorizer
dv = DictVectorizer(sparse=True)  # Reduce memory usage with sparse matrices
X_train = dv.fit_transform(features)

# Print dimensionality
print("Feature matrix shape:", X_train.shape[1])

Feature matrix shape: 518


### Q5. Answer: 41.99

In [46]:
# Define the target variable (trip duration in minutes)
target = 'duration'
y_train = df[target].values  # Extract target values as a numpy array

In [47]:
y_train

array([ 8.43333333,  6.31666667, 12.75      , ..., 24.51666667,
       13.        , 14.4       ])

In [48]:
from sklearn.linear_model import LinearRegression

# Initialize the model
lr = LinearRegression()

# Train the model on the training dataset
lr.fit(X_train, y_train)

LinearRegression()

In [49]:
# Predict the trip durations on the training dataset
y_pred = lr.predict(X_train)

In [50]:
from sklearn.metrics import mean_squared_error

# Calculate RMSE
rmse = mean_squared_error(y_train, y_pred, squared=False)  # squared=False gives RMSE
print(f"RMSE on Training Data: {rmse}")

RMSE on Training Data: 41.996492307453806


### Q6. Answer: 

In [51]:
# Validation feature matrix from Q4
print(f"Feature matrix shape (validation): {X_val.shape}")

# Validation target variable
print(f"Validation targets shape: {y_val.shape}")

NameError: name 'X_val' is not defined